In [2]:
import os
import fastmri
import numpy as np
import torch
import pytorch_lightning as pl
import PIL
import pandas as pd
import h5py
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
fastmri.__version__

'0.1.2a20210917b'

In [4]:
brain_mri_datapath = "/data/projects/recon/data/public/fastmri/brain/data/FLAIR/multicoil_train/"
mri_files = os.listdir(brain_mri_datapath)

annotation_path = "/home/lgdejong/scratch/project/RecSeg/annotations/"
annotation_file = "brain.csv" 
annotation_file_list = "brain_file_list.csv"

annatations_df = pd.read_csv(os.path.join(annotation_path, annotation_file), index_col=None)
file_list_df = pd.read_csv(os.path.join(annotation_path, annotation_file_list), header=None)
file_names = file_list_df[0].to_list()
file_names = [i+".h5" for i in file_names]

annotated_file_names = list(set(mri_files).intersection(set(file_names)))
annotated_file_names.sort()

In [5]:
annatations_df

,file,slice,study_level,x,y,width,height,label
0,file_brain_AXFLAIR_200_6002425,2,No,91.0,132.0,7.0,6.0,Nonspecific white matter lesion
1,file_brain_AXFLAIR_200_6002425,5,No,210.0,170.0,7.0,7.0,Nonspecific white matter lesion
2,file_brain_AXFLAIR_200_6002425,5,No,93.0,167.0,6.0,6.0,Nonspecific white matter lesion
3,file_brain_AXFLAIR_200_6002425,5,No,83.0,177.0,4.0,6.0,Nonspecific white matter lesion
4,file_brain_AXFLAIR_200_6002425,9,No,175.0,170.0,6.0,5.0,Nonspecific white matter lesion
...,...,...,...,...,...,...,...,...
8208,file_brain_AXT1POST_201_6002806,6,No,112.0,71.0,45.0,53.0,Mass
8209,file_brain_AXT1POST_201_6002806,6,No,99.0,69.0,65.0,96.0,Edema
8210,file_brain_AXT1POST_201_6002806,7,No,100.0,69.0,61.0,95.0,Edema
8211,file_brain_AXT1POST_201_6002806,8,No,105.0,86.0,50.0,71.0,Edema


In [15]:
def get_label(data_frame, file_name, slice=None):
    results = {}

    if file_name[-3:] == '.h5':
        file_name = file_name[:-3]

    file_label = data_frame.loc[data_frame['file'] == file_name]

    if slice is None:
        slice = list(file_label['slice'].unique())
        print(slice)
    else:
        slice = [slice]

    for i in slice:
        label = file_label.loc[file_label['slice'] == i].values.tolist()
        results[i] = label

    return results

def kspace2image(mri_kspace, slice=None):
    results = []

    if slice:
        slice = [slice]
    else:
        slice = [i for i in range(mri_kspace.shape[0])]

    for i in slice:
        kspace_slice = mri_kspace[i]
        kspace_slice = fastmri.data.transforms.to_tensor(kspace_slice)
        slice_image = fastmri.ifft2c(kspace_slice)
        slice_image = fastmri.complex_abs(slice_image) 
        if len(kspace_slice.shape) == 3:
            slice_image = fastmri.rss(slice_image, dim=0)
        slice_image = np.abs(slice_image.numpy())
        results.append(slice_image)

    if len(results) == 1:
        results = results[0]

    return results

label = get_label(annatations_df, annotated_file_names[0])
print(label.keys())
print(label[0])

[0, 1, 2, 3, 4, 7, 9]
dict_keys([0, 1, 2, 3, 4, 7, 9])
[['file_brain_AXFLAIR_200_6002429', 0, 'No', 187.0, 221.0, 14.0, 22.0, 'Nonspecific white matter lesion'], ['file_brain_AXFLAIR_200_6002429', 0, 'No', 109.0, 203.0, 30.0, 62.0, 'Nonspecific white matter lesion'], ['file_brain_AXFLAIR_200_6002429', 0, 'No', 182.0, 101.0, 9.0, 15.0, 'Nonspecific white matter lesion'], ['file_brain_AXFLAIR_200_6002429', 0, 'No', 134.0, 96.0, 8.0, 14.0, 'Nonspecific white matter lesion']]


In [16]:
def plot_bounding_box(image, labels):
    plotted_image = PIL.ImageDraw.Draw(image)
    for label in labels:
        _, _, _, x0, y0, w, h, label_txt = label
        x1 = x0 + w
        y1 = y0 + h
        plotted_image.rectangle(((x0,y0), (x1,y1)), outline="white")
        plotted_image.text((x0, max(0, y0 - 10)), label_txt, fill= "white")
    return np.array(image)